<a href="https://colab.research.google.com/github/kimx1201/movie_for_mood/blob/main/%EB%8B%A4%EC%9D%8C%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

     |████████████████████████████████| 911kB 5.3MB/s 


In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import numpy as np
import html5lib
from selenium.webdriver.common.keys import Keys
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
movie_list = pd.read_excel('/content/drive/MyDrive/project6/KOBIS_개봉일람.xlsx', parse_dates=['개봉일'])

In [ ]:
movie_list.to_csv("movie_list.csv", header=False, index=False)

In [ ]:
movie_list

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급
0,10577,초능력 에볼루션,존 라이드,2021-05-31,미국,35,SF,15세이상관람가
1,10753,스페이스 시그널,NaN,2021-05-31,미국,23,액션,12세이상관람가
2,12358,남친 옆에서 정사,이치고 타로와,2021-05-29,일본,1,멜로/로맨스,청소년관람불가
3,13472,발정난 그녀의 속사정,비바 곤조,2021-05-29,일본,1,멜로/로맨스,청소년관람불가
4,13501,밝히는 여자들:참을 수 없어,카부토가니,2021-05-29,일본,1,멜로/로맨스,청소년관람불가
...,...,...,...,...,...,...,...,...
17465,17207,여자들… 혹은 아이들 먼저…,마누엘 푀리에,NaT,프랑스,0,드라마,NaN
17466,17335,크리스마스 캐롤,지미 T. 무라카미,NaT,영국,0,애니메이션,12세관람가
17467,17359,통행증,베르트랑 타베르니에,NaT,프랑스,0,전쟁,NaN
17468,17382,패스트 푸드 패스트 우먼,아모스 콜렉,NaT,독일,0,코미디,기타


In [ ]:
for j in range(950, 1501, 50):
    
    movie_list = pd.read_csv('/content/drive/MyDrive/project6/movie_list.csv' %j)
    result = {}


    for title in tqdm(movie_list['영화명']):

        options = webdriver.ChromeOptions()
        options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
        options.add_argument('disable-gpu')    # GPU 사용 안함
        options.add_argument('lang=ko_KR')    # 언어 설정, options=options
        driver = webdriver.Chrome('chromedriver', options = options)
        driver.implicitly_wait(3)
        driver.get('https://www.daum.net/')
        driver.find_element_by_id('q').send_keys(title)
        driver.find_element_by_id('q').send_keys(Keys.RETURN)

        try:
            driver.find_element_by_xpath('//*[@id="movieMoreBtn"]').click()
        
            # time.sleep(1)
            # 새로운 탭으로 초점을 전환
            driver.switch_to_window(driver.window_handles[1])
            base_url = driver.current_url
            
            review_list = []
            for i in range(0, 10):
                time.sleep(1)
                an_url = '&type=netizen&page=%d' %(i+1)
                url = base_url +an_url

                res = requests.get(url)
                soup =  BeautifulSoup(res.text, 'html.parser')
                reviews = soup.find_all('p', {'class':'desc_review'})
                # print(reviews)

                for review in reviews :
                    review_list.append(review.get_text().strip())

            result[title] = review_list
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
            driver.close()
            
        except NoSuchElementException:
            driver.close()


    from pandas import DataFrame

    df = DataFrame(result.items(), columns = ['movie_id' , 'reviews'])
    df.to_csv('reviews/result_%d.csv' %(j/50), index =False, encoding='utf-8-sig') 

TypeError: ignored